## Search the Best Arbitrage in Bostrom Pools

In [1]:
import pandas as pd
from IPython.core.display import display, HTML

from src.data_extractors import get_pools, get_prices


MIN_ARBITRAGE_REVENUE = 0.02
H_START_AMOUNT = 1_000_000

#### Get Bostrom and Osmosis Pools Data

In [2]:
pools_df = get_pools(display_data=True)

network,id,type_id,balances,reserve_coin_denoms,swap_fee
bostrom,1,1,"[{'denom': 'boot', 'amount': '66237084030'}, {'...","[boot, hydrogen]",0.003
bostrom,2,1,"[{'denom': 'boot', 'amount': '16255814583'}, {'...","[boot, milliampere]",0.003
bostrom,3,1,"[{'denom': 'boot', 'amount': '21645465844'}, {'...","[boot, tocyb]",0.003
bostrom,4,1,"[{'denom': 'hydrogen', 'amount': '36376537278'}...","[hydrogen, tocyb]",0.003
bostrom,5,1,"[{'denom': 'hydrogen', 'amount': '448607469898'...","[hydrogen, milliampere]",0.003
bostrom,6,1,"[{'denom': 'hydrogen', 'amount': '493205757119'...","[hydrogen, millivolt]",0.003
bostrom,7,1,"[{'denom': 'hydrogen', 'amount': '21530822969'}...","[hydrogen, uosmo in bostrom]",0.003
bostrom,8,1,"[{'denom': 'hydrogen', 'amount': '35811572269'}...","[hydrogen, uatom in bostrom]",0.003
bostrom,9,1,"[{'denom': 'uosmo in bostrom', 'amount': '46491...","[uosmo in bostrom, uatom in bostrom]",0.003
bostrom,10,1,"[{'denom': 'milliampere', 'amount': '7216553'},...","[milliampere, millivolt]",0.003


#### Calculate Prices

In [3]:
price_df = get_prices(pools_df=pools_df, display_data=True)

,boot,millivolt,tocyb,milliampere,uosmo in bostrom,uosmo,boot in osmosis,uatom in osmosis,hydrogen,uatom in bostrom
boot,1,NaN,1.866515,7714.559345,NaN,NaN,1,NaN,0.447326,NaN
millivolt,NaN,1,NaN,0.132539,NaN,NaN,NaN,NaN,0.000008,NaN
tocyb,0.532548,NaN,1,NaN,NaN,NaN,NaN,NaN,0.236839,NaN
milliampere,0.000129,7.499745,NaN,1,NaN,NaN,NaN,NaN,0.000057,NaN
uosmo in bostrom,NaN,NaN,NaN,NaN,1,1,NaN,NaN,0.017659,3.905061
uosmo,NaN,NaN,NaN,NaN,1,1,0.035822,3.547396,NaN,NaN
boot in osmosis,1,NaN,NaN,NaN,NaN,27.748287,1,98.032168,NaN,NaN
uatom in osmosis,NaN,NaN,NaN,NaN,NaN,0.280208,0.01014,1,NaN,1
hydrogen,2.222114,129280.126452,4.196978,17401.570855,56.289095,NaN,NaN,NaN,1,218.090125
uatom in bostrom,NaN,NaN,NaN,NaN,0.254544,NaN,NaN,1,0.004558,1


In [4]:
coins_unique_set = set(price_df.columns)
result_list = []
for coin_1 in coins_unique_set:
    coin_1_amount = H_START_AMOUNT * price_df.loc[coin_1, 'hydrogen']
    if coin_1_amount > 0:
        for coin_2 in coins_unique_set.difference({coin_1}):
            coin_2_amount = coin_1_amount * price_df.loc[coin_2, coin_1]
            if coin_2_amount > 0:
                for coin_3 in coins_unique_set.difference({coin_1, coin_2}):
                    coin_3_amount = coin_2_amount * price_df.loc[coin_3, coin_2]
                    result = coin_3_amount * price_df.loc[coin_1, coin_3]
                    if result > coin_1_amount * (1 + MIN_ARBITRAGE_REVENUE):
                        result_list.append(
                                [[coin_1, coin_2, coin_3],
                                 round(result/coin_1_amount, 3)])
                    if coin_2_amount > 0:
                        for coin_4 in coins_unique_set.difference({coin_1, coin_2, coin_3}):
                            coin_4_amount = coin_3_amount * price_df.loc[coin_4, coin_3]
                            result = coin_4_amount * price_df.loc[coin_1, coin_4]
                            if result > coin_1_amount * (1 + MIN_ARBITRAGE_REVENUE):
                                result_list.append(
                                    [[coin_1, coin_2, coin_3, coin_4],
                                     round(result/coin_1_amount, 3)])

result_df =\
    pd.DataFrame(
        data=[[' -> '.join(coin for coin in item[0] + [item[0][0]]), item[1]] for item in result_list],
        columns=['arbitrage', 'revenue']).sort_values('revenue', ascending=False)
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = '{0:7,.2f}'.format
display(HTML(result_df.to_html(index=False, notebook=True, show_dimensions=False)))



arbitrage,revenue
uosmo in bostrom -> uosmo -> uatom in osmosis -> uatom in bostrom -> uosmo in bostrom,1.09
uatom in bostrom -> uosmo in bostrom -> uosmo -> uatom in osmosis -> uatom in bostrom,1.09
